Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 папкам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** item_category_id - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


# Задание 1

1. Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).
2. Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).

In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(dplyr)
df_sales_train <- read.csv("https://raw.githubusercontent.com/IYunet/VIM-2022/main/sales_train.csv", header = TRUE)
sort_summ <- df_sales_train %>% group_by(item_id) %>% summarise (total_sales = sum(item_cnt_day ),)
top10 <- head(sort_summ[order(sort_summ$total_sales, decreasing=TRUE),], n=10)
top10

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# A tibble: 10 × 2
   item_id total_sales
     <int>       <dbl>
 1   20949      187642
 2    2808       17245
 3    3732       16642
 4   17717       15830
 5    5822       14515
 6    3734       11688
 7    6675       10289
 8    3731       10099
 9    1855       10032
10   16787        9227


In [3]:
%%R
d_2013 <- df_sales_train[df_sales_train$date_block_num >= 0 & df_sales_train$date_block_num <= 11,]
d_2014 <- df_sales_train[df_sales_train$date_block_num >= 12 & df_sales_train$date_block_num <= 23,]
d_2015 <- df_sales_train[df_sales_train$date_block_num >= 24 & df_sales_train$date_block_num <= 33,]
check_id_each_dates <- d_2013[d_2013$item_id %in% c(d_2014$item_id ,d_2015$item_id ),]
top_new <- top10[top10$item_id %in% (check_id_each_dates$item_id),]
head(top_new, n = 9)

# A tibble: 9 × 2
  item_id total_sales
    <int>       <dbl>
1   20949      187642
2    2808       17245
3    3732       16642
4   17717       15830
5    5822       14515
6    3734       11688
7    6675       10289
8    1855       10032
9   16787        9227


# Задание 2

> При выполнении на pandas не используйте merge().

1. Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.
2. Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.

In [4]:
%%R
df_items <- read.csv("https://raw.githubusercontent.com/IYunet/VIM-2022/main/items.csv", header = TRUE)
df_item_categories <- read.csv("https://raw.githubusercontent.com/IYunet/VIM-2022/main/item_categories.csv", header = TRUE)
names(df_item_categories)[2] <-'category_id' # переимновал столбец для объединения
big_table <- df_sales_train %>% left_join(df_items,  by = "item_id") %>% left_join(df_item_categories, by = "category_id") 
filter_big_table <- big_table %>% group_by(category_id, item_category_name) %>% summarise(total_sales = sum(item_cnt_day ),)
sort_category <- filter_big_table[order(filter_big_table$total_sales, decreasing=TRUE ),]
second_task_min_category <- tail(sort_category, n=2)
second_task_min_category

`summarise()` has grouped output by 'category_id'. You can override using the
`.groups` argument.
# A tibble: 2 × 3
# Groups:   category_id [2]
  category_id item_category_name              total_sales
        <int> <chr>                                 <dbl>
1          10 " Game consoles - PS2"                    1
2          51 " Books - Cognitive literature"           1


In [5]:
%%R
second_task_max_category <- head(sort_category, n=1)
second_task_max_category

# A tibble: 1 × 3
# Groups:   category_id [1]
  category_id item_category_name total_sales
        <int> <chr>                    <dbl>
1          40 " Cinema - DVD"         634171


# дополнительное задание


In [6]:
%%R
# дополнительное задание 16
# Найдите категории, товары которой хуже всего продаются весной.
spring_2013 <- big_table[big_table$date_block_num >= 2 & big_table$date_block_num <= 4,]
spring_2014 <- big_table[big_table$date_block_num >= 14 & big_table$date_block_num <= 16,]
spring_2015 <- big_table[big_table$date_block_num >= 26 & big_table$date_block_num <= 28,]


In [7]:
%%R
categot_worst_spring_2013 <- spring_2013 %>% group_by(category_id) %>% summarise_if(is.numeric, min)
criteria <- min(categot_worst_spring_2013$item_cnt_day)
categot_worst_spring_2013[categot_worst_spring_2013$item_cnt_day == criteria,]
# 2013 худшая категория

# A tibble: 1 × 6
  category_id date_block_num shop_id item_id item_price item_cnt_day
        <int>          <int>   <int>   <int>      <dbl>        <dbl>
1          62              2       2    1442         28           -3


In [8]:
%%R
categot_worst_spring_2014 <- spring_2014 %>% group_by(category_id) %>% summarise_if(is.numeric, min)
criteria <- min(categot_worst_spring_2014$item_cnt_day)
categot_worst_spring_2014[categot_worst_spring_2014$item_cnt_day == criteria,]
# 2014 худшая категория

# A tibble: 2 × 6
  category_id date_block_num shop_id item_id item_price item_cnt_day
        <int>          <int>   <int>   <int>      <dbl>        <dbl>
1          30             14       2      28         14           -2
2          73             14       2     475        120           -2


In [9]:
%%R
categot_worst_spring_2015 <- spring_2015 %>% group_by(category_id) %>% summarise_if(is.numeric, min)
criteria <- min(categot_worst_spring_2015$item_cnt_day)
categot_worst_spring_2015[categot_worst_spring_2015$item_cnt_day == criteria,]
# 2015 худшая категория

# A tibble: 1 × 6
  category_id date_block_num shop_id item_id item_price item_cnt_day
        <int>          <int>   <int>   <int>      <dbl>        <dbl>
1          20             26       2    1389        231           -5


# Задание 3

1. Для категорий из задания 2 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами.
2. Найдите месяцы с минимальными и максимальными продажами для каждого товара из пункта 1.


In [10]:
%%R
summ_sells_in_shop <- df_sales_train %>% group_by(shop_id) %>% summarise(total_sales = sum(item_cnt_day ),)
shop_sort <- summ_sells_in_shop[order(summ_sells_in_shop$total_sales, decreasing=TRUE ),]
top_5_shops <- head(shop_sort, n = 5)

table_filter_min <- filter(big_table, category_id == second_task_min_category$category_id & shop_id == top_5_shops$shop_id)
table_min <- table_filter_min %>% group_by(item_id) %>% summarise(mean_sales = mean(item_cnt_day))
head(table_filter_min)
# пустой запрос ответ 3.1

[1] date               date_block_num     shop_id            item_id           
[5] item_price         item_cnt_day       item_name          category_id       
[9] item_category_name
<0 rows> (or 0-length row.names)


In [11]:
%%R
table_filter_max <- subset(big_table, shop_id %in% top_5_shops$shop_id & category_id == second_task_max_category$category_id) # почему разные результаты?
table_filter_max_1 <- big_table[big_table$category_id == second_task_max_category$category_id,]  
#table_filter_max <- table_filter_max_1[table_filter_max_1$shop_id %in% c(top_5_shops$shop_id),]

head(table_filter_max)
table_max <- table_filter_max %>% group_by(item_id) %>% summarise(mean_sales = mean(item_cnt_day))
table_max
# ответ 3.1 первые 6 (можно вывести больше)

# A tibble: 4,714 × 2
   item_id mean_sales
     <int>      <dbl>
 1       0          1
 2       2          1
 3       3          1
 4       4          1
 5       5          1
 6       6          1
 7       7          1
 8       8          1
 9       9          1
10      10          1
# … with 4,704 more rows


In [12]:
%%R
second_task_max_category$category_id

[1] 40


In [13]:
%%R
# отсортировать исходную таблицу по нужным id_items
#needed_items <- filter(df_sales_train, item_id == table_max$item_id) ?? почему не работает?
needed_items <- df_sales_train[df_sales_train$item_id %in% (table_max$item_id),]

#отсортировать по дате(номер месяца в порядке следования) и посчитать сумму продаж
sort_for_date <- needed_items %>% group_by(item_id, date_block_num) %>% summarise(total_sales = sum(item_cnt_day ))
min_sales <- sort_for_date %>% group_by(item_id) %>% summarise_if(is.numeric, min) # минимум
min_sales

`summarise()` has grouped output by 'item_id'. You can override using the
`.groups` argument.
# A tibble: 4,714 × 3
   item_id date_block_num total_sales
     <int>          <int>       <dbl>
 1       0             20           1
 2       2             19           1
 3       3             18           1
 4       4             20           1
 5       5             23           1
 6       6             18           1
 7       7             23           1
 8       8             19           1
 9       9             19           1
10      10             24           1
# … with 4,704 more rows


In [14]:
%%R
max_sales <- sort_for_date %>% group_by(item_id) %>% summarise_if(is.numeric, max) # максимум
max_sales

# A tibble: 4,714 × 3
   item_id date_block_num total_sales
     <int>          <int>       <dbl>
 1       0             20           1
 2       2             22           1
 3       3             19           1
 4       4             20           1
 5       5             23           1
 6       6             18           1
 7       7             23           1
 8       8             20           1
 9       9             19           1
10      10             24           1
# … with 4,704 more rows


In [15]:
%%R
# отсортировать по дате и посчитать сумму продаж
library(lubridate)
sort_for_date <- month(as.POSIXlt(needed_items$date, format="%d.%m.%Y")) 
copy_needed_items <- df_sales_train[df_sales_train$item_id %in% (table_max$item_id),]
copy_needed_items["date"] <- as.character(sort_for_date)
min_month <- copy_needed_items %>% group_by(item_id) %>% summarise_if(is.numeric, min)
head(min_month) # дата по месяцам спрятаны в date_block_num

R[write to console]: 
Attaching package: ‘lubridate’


R[write to console]: The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




# A tibble: 6 × 5
  item_id date_block_num shop_id item_price item_cnt_day
    <int>          <int>   <int>      <dbl>        <dbl>
1       0             20      54         58            1
2       2             19      54         58            1
3       3             18      54         58            1
4       4             20      54         58            1
5       5             23      54         28            1
6       6             18      54        100            1


In [16]:
%%R
max_month <- copy_needed_items %>% group_by(item_id) %>% summarise_if(is.numeric, max)
head(max_month) # дата по месяцам спрятаны в date_block_num

# A tibble: 6 × 5
  item_id date_block_num shop_id item_price item_cnt_day
    <int>          <int>   <int>      <dbl>        <dbl>
1       0             20      54         58            1
2       2             22      54         58            1
3       3             19      54        100            1
4       4             20      54         58            1
5       5             23      54         28            1
6       6             18      54        100            1


# Задание 4

1. Для категорий из задания 2 найдите среднюю и медианную цену по категории в пределах каждого месяца.
2. В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.

In [17]:
%%R
# сделать сортировку нужных категорий
requre_category_1 <- big_table[big_table$category_id %in% (second_task_min_category$category_id),]
requre_category_2 <- big_table[big_table$category_id %in% (second_task_max_category$category_id),]
requre_category_1 %>% group_by(category_id, date_block_num) %>% summarise(mean_price = mean(item_price))
# средняя цена

`summarise()` has grouped output by 'category_id'. You can override using the
`.groups` argument.
# A tibble: 2 × 3
# Groups:   category_id [2]
  category_id date_block_num mean_price
        <int>          <int>      <dbl>
1          10              3       1998
2          51             18        129


In [18]:
%%R
req_cat <- requre_category_2 %>% group_by(category_id, date_block_num) %>% summarise(mean_price = mean(item_price))
req_cat
# средняя цена

`summarise()` has grouped output by 'category_id'. You can override using the
`.groups` argument.
# A tibble: 34 × 3
# Groups:   category_id [1]
   category_id date_block_num mean_price
         <int>          <int>      <dbl>
 1          40              0       244.
 2          40              1       252.
 3          40              2       261.
 4          40              3       255.
 5          40              4       248.
 6          40              5       246.
 7          40              6       237.
 8          40              7       234.
 9          40              8       242.
10          40              9       248.
# … with 24 more rows


In [19]:
%%R
requre_category_1 %>% group_by(category_id, date_block_num) %>% summarise(median_price = median(item_price))
# медианная цена

`summarise()` has grouped output by 'category_id'. You can override using the
`.groups` argument.
# A tibble: 2 × 3
# Groups:   category_id [2]
  category_id date_block_num median_price
        <int>          <int>        <dbl>
1          10              3         1998
2          51             18          129


In [20]:
%%R
requre_category_2 %>% group_by(category_id, date_block_num) %>% summarise(median_price = median(item_price))
# медианная цена

`summarise()` has grouped output by 'category_id'. You can override using the
`.groups` argument.
# A tibble: 34 × 3
# Groups:   category_id [1]
   category_id date_block_num median_price
         <int>          <int>        <dbl>
 1          40              0          199
 2          40              1          203
 3          40              2          231
 4          40              3          199
 5          40              4          199
 6          40              5          199
 7          40              6          149
 8          40              7          149
 9          40              8          198
10          40              9          199
# … with 24 more rows


In [21]:
%%R
for(i in 2:33){
    if (req_cat$mean_price[i-1] > req_cat$mean_price[i] & req_cat$mean_price[i] < req_cat$mean_price[i+1]){
        print(req_cat$date_block_num[i])      
    }
    }
# 4.2

[1] 7
[1] 12
[1] 16
[1] 21
[1] 23
[1] 25
[1] 31
